# I (UN) decided to switch to using raw FF output for unet segmentation training. This script replaces all processed training files with raw

In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
from src.functions.utilities import path_leaf
pyo.init_notebook_mode()

### Get list of unet training directories

In [ ]:
from glob2 import glob

root = "/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/"
unet_root = os.path.join(root, "built_image_data", "unet_training", "*")
unet_dirs = sorted(glob(unet_root))
unet_dirs = [u for u in unet_dirs if os.path.isdir(u)]

### Put all processed images into subdirectories by date

In [ ]:
yolk_dir = unet_dirs[5]
annotation_list = glob(os.path.join(yolk_dir, "training", "annotations", "*"))
anno_names = [path_leaf(a)[:18] for a in annotation_list]

image_list = glob(os.path.join(yolk_dir, "training", "images", "*"))
image_names = [path_leaf(a)[:18] for a in image_list]

off_images = list(set(image_names) - set(anno_names))

In [ ]:
rm_indices = [i for i in range(len(image_names)) if image_names[i] in off_images]
rm_paths = [image_list[i] for i in rm_indices]
for rp in rm_paths:
    os.remove(rp)

In [ ]:
# import shutil
# from tqdm import tqdm

# image_root = os.path.join(root, "built_image_data", "stitched_FF_images", "")
# image_list = glob(image_root + "*")
# image_name_list = [path_leaf(im) for im in image_list]
# image_date_list = [im[:8] for im in image_name_list]
# image_stub_list = [im[9:] for im in image_name_list]

# date_list = np.unique(image_date_list)

# for date in tqdm(date_list):
#     date_dir = os.path.join(image_root, date)
#     if not os.path.isdir(date_dir):
#         os.makedirs(date_dir)

#     date_indices = np.where(np.asarray(image_date_list) == date)[0]

#     for di in date_indices:
#         from_path = image_list[di]
#         to_path = os.path.join(date_dir, image_stub_list[di])
#         shutil.move(from_path, to_path)

### iterate through training images and replace with contrast-adjusted versions

In [ ]:
import shutil
from tqdm import tqdm

image_root = os.path.join(root, "built_image_data", "stitched_FF_images")

for u, udir in enumerate(tqdm(unet_dirs[1:-1])):

    image_dir = os.path.join(udir, "training", "images", "")
    image_list = sorted(glob(image_dir + "*"))

    image_dir_new = os.path.join(udir, "training", "images_new", "")
    if not os.path.isdir(image_dir_new):
        os.makedirs(image_dir_new)

    for im in image_list:
        im_name = path_leaf(im)
        date = im_name[:8]
        stub = im_name[9:18]
        from_path = glob(os.path.join(image_root, date, stub + "*"))[0]
        to_path = os.path.join(image_dir_new, im_name)
        shutil.copyfile(from_path, to_path)

    # # copy images
    # for i in range(len(lb_names_u)):
    #     date_folder = lb_names_u[i][0:8]
    #     im_stub = lb_names_u[i][9:18]
    #     im_path_from = glob(os.path.join(image_root, date_folder, im_stub + "*"))[0]
    #     new_name = path_leaf(im_path_from)
    #     im_path_to = os.path.join(im_dir, date_folder + "_" + new_name)
    #     shutil.copyfile(im_path_from, im_path_to)
    
    
print(image_list)

In [ ]:
udir

### Split the live and dead labels into seperate sets

In [ ]:
import skimage.io as io

emb_dir = os.path.join(unet_root[:-1], "UNET_training_emb", "training") 
via_dir_lb = os.path.join(unet_root[:-1], "UNET_training_via", "training", "annotations") 
via_dir_im = os.path.join(unet_root[:-1], "UNET_training_via", "training", "images") 
if not os.path.isdir(os.path.join(via_dir_im)):
    os.makedirs(via_dir_lb)
    os.makedirs(via_dir_im)
    
mask_dir_lb = os.path.join(unet_root[:-1], "UNET_training_mask", "training", "annotations")
mask_dir_im = os.path.join(unet_root[:-1], "UNET_training_mask", "training", "images")
if not os.path.isdir(os.path.join(mask_dir_im)):
    os.makedirs(mask_dir_lb)
    os.makedirs(mask_dir_im)

label_list = sorted(glob(os.path.join(emb_dir, "annotations", "*")))
image_list = sorted(glob(os.path.join(emb_dir, "images", "*")))

for i in tqdm(range(len(image_list))):

    image_name = path_leaf(image_list[i])
    anno_path = glob(os.path.join(emb_dir, "annotations", image_name[0:19] + "*"))
    # loa
    im = io.imread(image_list[i])

    if len(anno_path) >0:
        lb = io.imread(anno_path[0])
         # make alternative labels
        lb_mask = lb.copy()
        lb_mask[lb_mask==2] = 1
    
        lb_via = lb.copy()
        lb_via[lb_via==1] = 0
        lb_via[lb_via==2] = 1

        io.imsave(os.path.join(mask_dir_lb, image_name), lb_mask, check_contrast=False)
        io.imsave(os.path.join(via_dir_lb, image_name), lb_via, check_contrast=False)
        
    # save
    io.imsave(os.path.join(mask_dir_im, image_name), im, check_contrast=False)
    io.imsave(os.path.join(via_dir_im, image_name), im, check_contrast=False)

In [ ]:
import skimage.io as io

emb_dir = os.path.join(unet_root[:-1], "UNET_training_emb", "training") 
via_dir_lb = os.path.join(unet_root[:-1], "UNET_training_via", "training", "annotations") 
via_dir_im = os.path.join(unet_root[:-1], "UNET_training_via", "training", "images") 
if not os.path.isdir(os.path.join(via_dir_im)):
    os.makedirs(via_dir_lb)
    os.makedirs(via_dir_im)
    
mask_dir_lb = os.path.join(unet_root[:-1], "UNET_training_mask", "training", "annotations")
mask_dir_im = os.path.join(unet_root[:-1], "UNET_training_mask", "training", "images")
if not os.path.isdir(os.path.join(mask_dir_im)):
    os.makedirs(mask_dir_lb)
    os.makedirs(mask_dir_im)

label_list = sorted(glob(os.path.join(emb_dir, "annotations", "*")))
image_list = sorted(glob(os.path.join(emb_dir, "images", "*")))

for i in tqdm(range(len(image_list))):

    image_name = path_leaf(image_list[i])
    anno_path = glob(os.path.join(emb_dir, "annotations", image_name[0:19] + "*"))
    # loa
    im = io.imread(image_list[i])

    if len(anno_path) >0:
        lb = io.imread(anno_path[0])
         # make alternative labels
        lb_mask = lb.copy()
        lb_mask[lb_mask==2] = 1
    
        lb_via = lb.copy()
        lb_via[lb_via==1] = 0
        lb_via[lb_via==2] = 1

        io.imsave(os.path.join(mask_dir_lb, image_name), lb_mask, check_contrast=False)
        io.imsave(os.path.join(via_dir_lb, image_name), lb_via, check_contrast=False)
        
    # save
    io.imsave(os.path.join(mask_dir_im, image_name), im, check_contrast=False)
    io.imsave(os.path.join(via_dir_im, image_name), im, check_contrast=False)

In [ ]:
/media/nick/hdd02/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/built_image_data/unet_training/UNET_training_emb/training/annotations/